In [ ]:
! pip install langchain_community tiktoken langchain-google-genai langchainhub chromadb langchain

In [12]:
import os
from dotenv import load_dotenv
load_dotenv() 
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] =  os.environ.get('LANGCHAIN_API_KEY')

In [13]:
os.environ['GEMINI_API_KEY'] = os.environ.get('GEMINI_API_KEY')

## 1.1 Multi Query
### <i>a technique where an initial user query is transformed into multiple, distinct queries, typically to enhance the retrieval of relevant information. </i>

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

In [15]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings 
from langchain_community.vectorstores import Chroma
import os

api_key = os.environ.get("GEMINI_API_KEY")


vectorstore = Chroma.from_documents(
    documents=splits, 
    embedding=GoogleGenerativeAIEmbeddings(model="text-embedding-004", google_api_key=api_key)
)
retriever = vectorstore.as_retriever()

In [17]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

template = """You are an AI language model assistant. Your task is to generate five 
different versions of the given user question to retrieve relevant documents from a vector 
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search. 
Provide these alternative questions separated by newlines. Original question: {question}"""
prompt_perspectives = ChatPromptTemplate.from_template(template)

generate_queries = (
    prompt_perspectives 
    | ChatGoogleGenerativeAI(model="gemini-2.5-flash",google_api_key=api_key, temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

In [ ]:
from langchain.load import dumps, loads

def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

# Retrieve
question = "What is task decomposition for LLM agents?"
retrieval_chain = generate_queries | retriever.map() | get_unique_union
docs = retrieval_chain.invoke({"question":question})
len(docs)

In [19]:
from operator import itemgetter
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.runnables import RunnablePassthrough

#RAG
template = """Answer the following quetsion based on this context:

{context}

Question:{question}
"""

prompt = ChatPromptTemplate.from_template(template)

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",google_api_key=api_key,temperature=0)
final_rag_chain = (
    {"context": retrieval_chain, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM agents is the process where the agent breaks down large, complicated tasks into smaller, more manageable subgoals or steps. This enables the efficient handling of complex tasks and allows the agent to plan ahead.\n\nCommon techniques for task decomposition include:\n*   **Chain of Thought (CoT) prompting**: Instructing the LLM to "think step by step" to decompose hard tasks into simpler steps.\n*   **Tree of Thoughts**: Extending CoT by exploring multiple reasoning possibilities at each step, generating multiple thoughts per step in a tree structure.\n\nTask decomposition can be achieved by:\n*   LLM with simple prompting (e.g., "Steps for XYZ.\\\\n1.", "What are the subgoals for achieving XYZ?").\n*   Using task-specific instructions (e.g., "Write a story outline.").\n*   Human inputs.'

## 1.2 RAG FUSION
### <i>RAG-Fusion is an advanced information retrieval technique that enhances Retrieval-Augmented Generation (RAG) by generating multiple queries from an initial user query, retrieving relevant documents for each, and then using Reciprocal Rank Fusion (RRF) to combine and re-rank the results, ultimately producing more accurate and comprehensive answers from a large language model (LLM).</i>

In [20]:
from langchain.prompts import ChatPromptTemplate

template = """You are a helpful assistant who generates multiple search queries based on a single input query.\n 
Generate multiple search queries related to : {question} \n
Output (4 queries) :"""

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

In [21]:
import os
api_key = os.environ.get("GEMINI_API_KEY")

In [22]:
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

generate_queries = (
    prompt_rag_fusion
    | ChatGoogleGenerativeAI(model="gemini-2.5-flash",google_api_key=api_key,temperature=0)
    | StrOutputParser()
    | (lambda x : x.split("\n"))
)

In [23]:
from langchain.load import dumps,loads

def reciprocal_rank_fusion(results : list[list],k=60):
    """Reciprocal_rank_fusion that takes multiple list of ranked documents
    and an optional paramter k used in RRF formula"""
    
    fused_scores = {}
    
    for docs in results:
        for rank,doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1/(rank+k)
    reranked_results = [
        (loads(doc),score)
        for doc,score in sorted(fused_scores.items(),key=lambda x : x[1],reverse=True)
    ]
    
    return reranked_results

retrieval_chain_rag_fusion = generate_queries | retriever.map() | reciprocal_rank_fusion
docs = retrieval_chain_rag_fusion.invoke({"question": question})
len(docs)
    

10

In [24]:
from langchain_core.runnables import RunnablePassthrough

# RAG
template = """Answer the following question based on this context:

{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    {"context": retrieval_chain_rag_fusion, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"question":question})

'Task decomposition for LLM agents is the process of breaking down large, complex tasks into smaller, more manageable subgoals or steps. This is a crucial part of the agent\'s planning component, enabling it to handle complex tasks efficiently and enhance its performance.\n\nKey aspects of task decomposition include:\n*   **Purpose:** It transforms big tasks into multiple manageable tasks, allowing the model to utilize more computation and providing insight into its thinking process.\n*   **Techniques:**\n    *   **Chain of Thought (CoT):** A standard prompting technique where the model is instructed to "think step by step" to decompose hard tasks into simpler ones.\n    *   **Tree of Thoughts (ToT):** An extension of CoT that explores multiple reasoning possibilities at each step, generating multiple thoughts per step and creating a tree structure.\n*   **Methods of Implementation:**\n    *   Using the LLM itself with simple prompts (e.g., "Steps for XYZ," "What are the subgoals for a

## 1.3 DECOMPOSITION
### <i>query decomposition is a query transformation technique that addresses complex user queries by breaking them down into simpler, more manageable sub-questions or steps<i>


In [25]:
from langchain.prompts import ChatPromptTemplate

# Decomposition
template = """You are a helpful assistant that generates multiple sub-questions related to an input question. \n
The goal is to break down the input into a set of sub-problems / sub-questions that can be answers in isolation. \n
Generate multiple search queries related to: {question} \n
Output (3 queries):"""
prompt_decomposition = ChatPromptTemplate.from_template(template)

In [26]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

# LLM
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash",google_api_key=api_key,temperature=0)

# Chain
generate_queries_decomposition = ( prompt_decomposition | llm | StrOutputParser() | (lambda x: x.split("\n")))

# Run
question = "What are the main components of an LLM-powered autonomous agent system?"
questions = generate_queries_decomposition.invoke({"question":question})

Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised InternalServerError: 500 Internal error encountered..


In [27]:
questions

['Here are 3 search queries related to the main components of an LLM-powered autonomous agent system:',
 '',
 '1.  "Architecture of LLM-powered autonomous agents"',
 '2.  "Key components of LLM agent systems"',
 '3.  "LLM agent design patterns and modules"']

In [ ]:
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

# RAG prompt
prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question,prompt_rag,sub_question_generator_chain):
    """RAG on each sub-question"""
    
    # Use our decomposition / 
    sub_questions = sub_question_generator_chain.invoke({"question":question})
    
    # Initialize a list to hold RAG chain results
    rag_results = []
    
    for sub_question in sub_questions:
        
        # Retrieve documents for each sub-question
        retrieved_docs = retriever.get_relevant_documents(sub_question)
        
        # Use retrieved documents and sub-question in RAG chain
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs, 
                                                                "question": sub_question})
        rag_results.append(answer)
    
    return rag_results,sub_questions

# Wrap the retrieval and RAG process in a RunnableLambda for integration into a chain
answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

In [29]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"Question {i}: {question}\nAnswer {i}: {answer}\n\n"
    return formatted_string.strip()

context = format_qa_pairs(questions, answers)

# Prompt
template = """Here is a set of Q+A pairs:

{context}

Use these to synthesize an answer to the question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

final_rag_chain.invoke({"context":context,"question":question})

'An LLM-powered autonomous agent system is primarily controlled by a Large Language Model (LLM) at its core. These systems are typically composed of several key components that enable them to perceive, plan, act, and learn.\n\nThe main components include:\n\n1.  **Planning:** This component allows the agent to break down complex tasks into manageable steps, strategize, and refine its actions. It often involves mechanisms for self-reflection, enabling the agent to learn from past experiences and adjust its future plans, as seen in approaches like ReAct (synergizing reasoning and acting) and Reflexion.\n2.  **Memory:** Essential for retaining and recalling information over various durations.\n    *   **Short-Term Memory:** For immediate context and ongoing tasks.\n    *   **Long-Term Memory (LTM):** Stores information for extended periods (days to decades) with essentially unlimited capacity. For agents, this is often implemented as an external vector store or a "memory stream" that reco

## 1.4 STEP BACK
### <i> a technique where the AI generates a more general, higher-level version of the original user question.</i>

## 1.5 HYDE
### <i>a query transformation technique that improves retrieval by generating a hypothetical, well-structured document in response to a user's query. This "fake" document, created by a large language model (LLM), is then embedded and used for similarity search against the knowledge base's document embeddings, bridging the gap between short, informal queries and the longer, formal format of actual documents. </i>